In [1]:
import os
import pickle

data_folder = '/home/mcwave/code/automath/atp/datasets/provability/mathlib4_states_w_proof/'
file_names = os.listdir(data_folder)

data = []

count = 0
for file_name in file_names:
    if not file_name.endswith("pkl"):
        continue
    if not 'Algebra' in file_name:
        continue
    count += 1
    if count <= 5:
        continue
    print("Loading", file_name)
    file_path = os.path.join(data_folder, file_name)
    fin = open(file_path, 'rb')
    while True:
        try:
            pair = pickle.load(fin)
            data.append(pair) #(pair[1][0], pair[1][2][0]))
        except:
            break
    break

print(len(data), "examples loaded")

Loading Mathlib__Algebra__Ring__Subring__Pointwise.lean.pkl
12680 examples loaded


In [2]:
data[1]

(('https://github.com/leanprover-community/mathlib4',
  '27c6744e1c0e25d676be5eb252cd4b6d30c6acc7',
  'Mathlib/Algebra/Ring/Subring/Pointwise.lean',
  'Subring.pointwise_smul_def'),
 ('case h\nM : Type u_1\nR : Type u_2\ninst✝² : Monoid M\ninst✝¹ : Ring R\ninst✝ : MulSemiringAction M R\na : M\nS : Subring R\nnvar0 : R\n⊢ nvar0 ∈ SMul.smul a S ↔ nvar0 ∈ map (MulSemiringAction.toRingHom M R a) S',
  2,
  ['simp_rw [mem_map]', 'exact']))

In [3]:
from utils.lean_math_utils import *
from utils.lean_theorem_utils import *

def count_lines(string):
    # Split the string into lines
    lines = string.splitlines()
    # Count the number of lines
    return len(lines)

def extract_first_case(state_pp):
    state_pp = state_pp.strip()
    if not state_pp.startswith('case'):
        return state_pp
    lines = state_pp.split('\n')
    first_case = []
    for line in lines[1:]:
        if line.strip().startswith('case'):
            break
        if line.strip() != '':
            first_case.append(line)
    return '\n'.join(first_case)


# Params:
#   hyp: tuple(name, type)
#   tactics: list(tactic)
def is_hypothesis_useful(hyp, tactics):
    for tactic in tactics:
        tokens = tokenize_lean_tactic(tactic)
        if hyp[0] in tokens:
            idx = tokens.index(hyp[0])
            if idx > 0:
                if hyp[0].startswith('h'):
                    return True
                if tokens[idx - 1] == 'exact':
                    return True
                if tokens[idx - 1] == 'at':
                    return True
                if tokens[idx - 1] == '[':
                    return True
                if idx < len(tokens) - 1 and tokens[idx + 1] == ']':
                    return True
                for operator in TargetNode.operators:
                    if operator in hyp[1]:
                        return True
    return False

def create_hypothesis_predict_data(raw_state_pp, tactics, theorem_name):
    is_case = raw_state_pp.strip().startswith('case')
    state_pp = extract_first_case(raw_state_pp)
    if is_case and count_lines(state_pp) < count_lines(raw_state_pp) - 2:
        tactics = tactics[0:1]
    #
    premise = Premise()
    premise.theorem_name = theorem_name
    premise.parse_state(state_pp)
    #
    useful_hypotheses, useless_hypotheses = [], OrderedDict()
    for hyp in premise.hypotheses.items():
        useful = is_hypothesis_useful(hyp, tactics)
        if useful:
            #print("YES:", hyp)
            useful_hypotheses.append(hyp)
        else:
            #print("NO :", hyp)
            useless_hypotheses[hyp[0]] = hyp[1]
    premise.hypotheses = useless_hypotheses
    return premise, useful_hypotheses

idx = 120

state_pp = data[idx][1][0]
tactics = data[idx][1][2]
theorem_name = data[idx][0][3]

print("STATE_PP:\n" + state_pp)

print("TACTICS:\n" + "\n".join(tactics))

premise, useful_hypotheses = create_hypothesis_predict_data(state_pp, tactics, theorem_name)

print("STATE_PP:\n" + premise.to_theorem_code())
print("\nHYPOTHESES:\n", useful_hypotheses)


STATE_PP:
case h
M : Type u_1
R : Type u_2
inst✝² : Monoid M
inst✝¹ : Ring R
inst✝ : MulSemiringAction M R
a : M
S : Subring R
nvar0 : R
⊢ nvar0 ∈ map (MulSemiringAction.toRingHom M R a) S ↔ nvar0 ∈ a • S
TACTICS:
symm
simp [eq_comm (a := a)]
cases S
rw [smul_neg]
rw [← eq_f₀']
STATE_PP:
theorem Subring.pointwise_smul_def (M: Type u_1) (R: Type u_2) (inst✝²: Monoid M) (inst✝¹: Ring R) (inst✝: MulSemiringAction M R) (a: M) (S: Subring R) (nvar0: R) : nvar0 ∈ map (MulSemiringAction.toRingHom M R a) S ↔ nvar0 ∈ a • S :=

HYPOTHESES:
 []


In [4]:
from datasets import Dataset

MIN_LENGTH = 4

state_pps = []
target_hyps = []
seen_hashes = set()
fin = open('/home/mcwave/code/axiomatization/datasets/mathlib4_algebra_states_w_proof_hyp_pred.pkl', 'rb')

while True:
    try:
        premise, hypotheses = pickle.load(fin)
        state_pp = premise.to_theorem_code()
        target_hyp = str([x[1] for x in hypotheses])
        hash_value = hash(state_pp + '|' + target_hyp)
        if hash_value in seen_hashes:
            continue
        else:
            seen_hashes.add(hash_value)
        #data.append((state_pp, target_hyp))
        if len(state_pp) < 4 or len(target_hyp) < 4:
            continue
        state_pps.append(state_pp)
        target_hyps.append(target_hyp)
    except:
        break
    
fin.close()

print(len(state_pps))

# ds = Dataset.from_dict({'state_pp': state_pps, 'target_hyp':target_hyps})
# tmp = ds.train_test_split(test_size=0.01)
# raw_train_dataset = tmp['train']
# raw_test_dataset = tmp['test']

281950


In [6]:
import torch
from transformers import AutoTokenizer
from datasets import Dataset

MAX_LENGTH = 256

# Assuming you have two lists: instructions and responses
instructions = state_pps #["Instruction 1", "Instruction 2"]
responses = target_hyps #["Response 1", "Response 2"]
#instructions = ["Instruction 1", "Instruction 2"]
#responses = ["Response 1", "Response 2"]

# Initialize tokenizer and model

tokenizer = AutoTokenizer.from_pretrained("datasets/text_for_tokenization/mathlib4_20240617_bpe_tokenizer")
# Define the tokens
sep_token = "<sep>"
pad_token = "<pad>"

# Set the sep_token and pad_token
tokenizer.sep_token = sep_token
tokenizer.pad_token = pad_token
#tokenizer.add_special_tokens({'sep_token': '[SEP]'})


# Function to tokenize and prepare the data
def prepare_data(examples):
    # Concatenate instruction and response with a separator
    full_texts = [f"{instruction} <sep> {response}" for instruction, response in zip(examples['instruction'], examples['response'])]
    
    # Tokenize the full texts
    encodings = tokenizer(full_texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors='pt')
    #print(encodings)
    
    # Create attention masks: 1 for response tokens, 0 for instruction tokens and padding
    attention_masks = []
    labels = []
    
    for input_ids in encodings['input_ids']:
        attention_mask = torch.zeros_like(input_ids)
        label = input_ids.clone()
        
        pad_token_idx = (input_ids == tokenizer.pad_token_id).nonzero()
        end_idx = pad_token_idx[0].item() if len(pad_token_idx) > 0 else len(input_ids)
        sep_token_idx = (input_ids == tokenizer.sep_token_id).nonzero()
        #print("sep_token_idx:", sep_token_idx)
        if len(sep_token_idx) == 0:
            sep_token_idx = 0
        else:
            sep_token_idx = sep_token_idx.item()

        attention_mask[0:end_idx] = 1
        attention_masks.append(attention_mask)
        
        label[0:sep_token_idx+1] = -100
        labels.append(label)
    
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': torch.stack(attention_masks),
        'labels': torch.stack(labels)
    }

# Create the Hugging Face dataset
dataset = Dataset.from_dict({
    'instruction': instructions,
    'response': responses
})

# Apply the tokenization and preparation function
tokenized_dataset = dataset.map(
    prepare_data,
    batched=True
    #remove_columns=dataset.column_names
)

Map:   0%|          | 0/281950 [00:00<?, ? examples/s]

In [20]:
len(tokenized_dataset[100000]['input_ids'])

256

In [21]:
tmp = tokenized_dataset.train_test_split(test_size=0.02)
tokenized_train = tmp['train']
tokenized_test = tmp['test']

In [22]:
from transformers import GPTNeoForCausalLM, Trainer, TrainingArguments

model_name = "xhyi/PT_GPTNEO350_ATG"  # You can change this to a larger GPTNeo model if needed
model = GPTNeoForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

/home/mcwave/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Embedding(15000, 1024)

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="datasets/predict-hyp-gptneo-350m",
    evaluation_strategy="steps", #"epochs"
    learning_rate=2e-5,  # PAY ATTENTION TO LEARNING RATE!
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    fp16=True,
    save_steps=5000,
    eval_steps=5000,
    logging_steps=5000,
    save_total_limit=2,
    load_best_model_at_end=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
)

#cp_path = 'gpt-neo-350m-202310/checkpoint-525000'

trainer.train()

In [32]:
# num_epochs = 4
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0

#     for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         labels = batch["labels"].to(device)

#         outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
    
#     # Save the model after each epoch
#     save_path = f"/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_{epoch + 1}"
#     os.makedirs(save_path, exist_ok=True)
#     model.save_pretrained(save_path)
    
#     avg_train_loss = total_loss / len(train_dataloader)
#     avg_test_loss = evaluate(model, test_dataloader, device)

#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     print(f"Average Train Loss: {avg_train_loss:.4f}")
#     print(f"Average Test Loss: {avg_test_loss:.4f}")

Epoch 1/4:  36%|███████████████████████████▌                                                 | 106651/298283 [21:38:44<38:53:35,  1.37it/s]


KeyboardInterrupt: 

In [36]:
avg_test_loss = evaluate(model, test_dataloader, device)

In [35]:
# save_path = f"/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_{epoch + 1}"
# print(save_path)
# os.makedirs(save_path, exist_ok=True)
# model.save_pretrained(save_path)

/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_1


In [6]:
save_path = f"/home/mcwave/code/axiomatization/datasets/hyp_pred_byt5_small/trained_model_epoch_1"
model = AutoModelForSeq2SeqLM.from_pretrained(save_path)

In [21]:
test_case = tokenized_test[7]
input_ids = torch.tensor(test_case['input_ids']).unsqueeze(0).to('cuda')  # Add batch dimension
print("inputs:", tokenizer.decode(test_case['input_ids'], skip_special_tokens=True))
labels = torch.tensor(test_case['labels']).to('cuda')
print("labels:", tokenizer.decode(labels, skip_special_tokens=True))

# Generate output
with torch.no_grad():
    outputs = model.generate(input_ids)

# Decode the generated output and the true labels
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
true_text = tokenizer.decode(labels, skip_special_tokens=True)

# Compare the results
print("Generated:", generated_text)
print("True:", true_text)

inputs: theorem Function.Exact.apply_apply_eq_zero (R: Type u_1) (M: Type u_2) (M': Type u_3) (N: Type u_4) (N': Type u_5) (P: Type u_6) (P': Type u_7) (f: M → N) (g: N → P) (inst✝: Zero P) (x: M) : ¬¬¬¬¬Exact f g :=
labels: ['¬¬¬¬¬g (f x) = 0']


/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated: _ (S__u_7)) (G) (R
True: ['¬¬¬¬¬g (f x) = 0']


In [10]:
labels

tensor([94, 42, 74, 42, 96,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 